In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [ ]:
dataset_path = os.getcwd()
dataset_path += "\\A_Z Handwritten Data.csv"
dataset_path_path = os.path.abspath(dataset_path)

df = pd.read_csv(dataset_path)

# print first 5 rows
df.head(5)

## DATA PREPROCESSING

In [ ]:
# extract class labels into separate dataframe
class_column = df['0']
del df['0']

In [ ]:
class_column_letters = class_column.replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25], ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'])

In [ ]:
alphabet = ['O', 'S', 'U', 'C' , 'T' , 'P' , 'N', 'A', 'M', 'L', 'R', 'E', 'Y', 'W', 'D', 'B', 'J', 'H', 'X', 'Z', 'Q', 'G', 'K', 'V', 'F', 'I']
counts = class_column_letters.value_counts()

fig, ax = plt.subplots(1, 1, figsize=(24, 16))
ax.barh(alphabet, counts, color='green',)

plt.xlabel('Count')
plt.ylabel('Letter')
plt.yticks(fontsize=14)
plt.title('Occurence of each letter in the dataset.', fontsize=24)
plt.grid()
plt.show()